!pip install nlpaug

In [1]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug

import pandas as pd
import sys
import csv

In [2]:
### Char Augmenter
# Optical character recognition
aug_char_OCR = nac.OcrAug()
# Keyboard
aug_char_Keyboard = nac.KeyboardAug()

### Word Augmenter
# Spelling
aug_word_Spelling = naw.SpellingAug()
#aug_word_WordEmbedng = naw.WordEmbsAug(model_type = 'word2vec', model_path=model_dir+'GoogleNews-vectors-negative300.bin', action="insert")
#aug_word_TfIdfAug = naw.TfIdfAug(model_path = os.environ.get("MODEL_DIR"), action = 'insert')
aug_word_ContextualWordEmbs = naw.ContextualWordEmbsAug(model_path = 'bert-base-uncased', action = 'insert')
aug_word_Synonym = naw.SynonymAug(aug_src = 'wordnet')
aug_word_Antonym = naw.AntonymAug()
aug_word_Split = naw.SplitAug()

Downloading: 100%|██████████| 570/570 [00:00<00:00, 571kB/s]
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


# 원본 데이터 정제

In [3]:
def refine_data(dataset_name):
    # 원본 HADOOP csv데이터 불러와서 정제하기
    issueReport = pd.read_csv("../Dataset/Deepsoft_IssueData/{}.csv".format(dataset_name))
    issueOnehot = issueReport.drop(columns = ['issuekey', 'title', 'description', 'component'])

    issueLabel = []
    for i in range(len(issueOnehot)):
        issueLabel.append(list(issueOnehot.iloc[i]))

    # make 'data' value
    refined_data = issueReport

    issueText = pd.Series(list(issueReport["title"] + ' ' + issueReport['description']), index = refined_data.index)
    refined_data['text'] = issueText

    refined_data['labels'] = pd.Series(issueLabel, index = refined_data.index)

    refined_data = refined_data.drop(columns = ['issuekey', 'title', 'description', 'component'])
    refined_data = pd.DataFrame(data = {'text':issueText, 'labels': issueLabel})
    return refined_data

In [4]:
def list_data(refined_data):
    # 정제된 데이터 text 리스트화
    data_list = []
    len(refined_data['text'])
    for x in range(len(refined_data['text'])):
        data_list.append(refined_data['text'][x])
    return data_list

#AUG_INTERVAL = 2


# 원본 데이터 augmentation
f = open('HADOOP_aug_mul3.csv', 'a', newline='', encoding='utf-8')
wr = csv.writer(f)
wr.writerow(data)
for x in range(len(data)):
    wr.writerow([data['text'][x], data['labels'][x]])

for line in range(AUG_RANGE):
    augmented_Embeded = aug_Embeded.augment(data['text'][line])
    wr.writerow([augmented_Embeded, data['labels'][line]])
f.close()

In [5]:
def augment_data(dataset_name, dataset, aug_target, augmenter, augmenter_name, augment_times = 6):
    # augmentation 데이터 추가
    if augment_times <= 0:
        return

    aug_range = len(dataset)
    f = open('../Dataset/Deepsoft_IssueData_AUG/{}_{}_{}.csv'.format(dataset_name, aug_target, augmenter_name), 'w', newline='', encoding='utf-8')
    wr = csv.writer(f)
    wr.writerow(dataset)
    for x in range(len(dataset)):
        wr.writerow([dataset['text'][x], dataset['labels'][x]])
    
    for x in range(augment_times): # aug multip
        for line in range(aug_range):
            augmented_Embeded = augmenter.augment(dataset['text'][line])
            wr.writerow([augmented_Embeded, dataset['labels'][line]])
    
    f.close()

In [6]:
# refine_data, augment_data

dataset_HADOOP = refine_data("HADOOP")
dataset_FCREPO = refine_data("FCREPO")
dataset_ISLANDORA = refine_data("ISLANDORA")

In [7]:
dataset_FCREPO

,text,labels
0,External triplestore with additional capabilit...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,BDef and BMech Modification : with Integrity A...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Logging Refactoring : consistency and enable e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Proper Architecture of Multiple Triplestore Op...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Content Model Architecture (CMA) Date: 2006-09...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1648,Update fcrepo-indexing-solr to use fcrepo-ldpa...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1649,Remove extra pom file Remove unneeded pom file...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1650,Importer needs to create repository resources ...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1651,Ensure Link Headers not too long This task is ...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
augment_data("HADOOP", dataset_HADOOP, 'char', aug_char_OCR, "OCR")

In [9]:
augment_data("HADOOP", dataset_HADOOP, 'char', aug_char_Keyboard, "Keyboard")

In [10]:
augment_data("HADOOP", dataset_HADOOP, 'word', aug_word_Spelling, "Spelling")

In [11]:
augment_data("HADOOP", dataset_HADOOP, 'word', aug_word_ContextualWordEmbs, "ContextualWordEmbs")

Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


In [12]:
augment_data("HADOOP", dataset_HADOOP, 'word', aug_word_Synonym, "Synonym")

In [13]:
augment_data("HADOOP", dataset_HADOOP, 'word', aug_word_Antonym, "Antonym")

In [14]:
augment_data("HADOOP", dataset_HADOOP, 'word', aug_word_Split, "Split")

# FCREPO

In [15]:
augment_data("FCREPO", dataset_FCREPO, 'char', aug_char_OCR, "OCR")

In [16]:
augment_data("FCREPO", dataset_FCREPO, 'char', aug_char_Keyboard, "Keyboard")

In [17]:
augment_data("FCREPO", dataset_FCREPO, 'word', aug_word_Spelling, "Spelling")

In [18]:
augment_data("FCREPO", dataset_FCREPO, 'word', aug_word_ContextualWordEmbs, "ContextualWordEmbs")

In [19]:
augment_data("FCREPO", dataset_FCREPO, 'word', aug_word_Synonym, "Synonym")

In [20]:
augment_data("FCREPO", dataset_FCREPO, 'word', aug_word_Antonym, "Antonym")

In [21]:
augment_data("FCREPO", dataset_FCREPO, 'word', aug_word_Split, "Split")

# ISLANDORA

In [22]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'char', aug_char_OCR, "OCR")

In [23]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'char', aug_char_Keyboard, "Keyboard")

In [24]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'word', aug_word_Spelling, "Spelling")

In [25]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'word', aug_word_ContextualWordEmbs, "ContextualWordEmbs")

In [26]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'word', aug_word_Synonym, "Synonym")

In [27]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'word', aug_word_Antonym, "Antonym")

In [28]:
augment_data("ISLANDORA", dataset_ISLANDORA, 'word', aug_word_Split, "Split")